In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
import os
from torchvision import models, transforms
import torch.optim as optim

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
batch_size = 128

In [5]:
#train and test data directory
train_dir = "train"
test_dir = "test"

In [7]:
# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Compose([transforms.Resize((64,64))])])

In [8]:
#load the train and test data
dataset = ImageFolder(train_dir,transform=transform)
test_dataset = ImageFolder(test_dir,transform=transform)

In [11]:
#initiate data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [20]:

dataloaders = [train_loader, test_loader]

In [12]:
classes = os.listdir("test")
classes

['21to30', '11to20', '61plus', '41to60', '0to10', '31to40']

In [15]:
model = models.resnet50(pretrained=True).to(device)

/home/alex/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/alex/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/alex/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [16]:
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 6)).to(device)

# need to add an output of 6 classes

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [21]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in [0,1]:
            if phase == 0:
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 0:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = running_corrects.double() / len(dataloaders[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [22]:
model_trained = train_model(model, criterion, optimizer, num_epochs=3)

Epoch 1/3
----------
0 loss: 172.2993, acc: 58.4970
1 loss: 158.5798, acc: 62.8421
Epoch 2/3
----------
0 loss: 157.3628, acc: 64.1078
1 loss: 153.2516, acc: 64.0526
Epoch 3/3
----------
0 loss: 153.2865, acc: 65.2874
1 loss: 151.5294, acc: 64.1053


### Save and Load Model

torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')

model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 6)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))